# 資料抓取與儲存成 DB

In [16]:
import time
import sqlite3
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ===== 基本設定 =====
BASE = "https://sea.cc.ntpu.edu.tw/pls/dev_stud/"
LIST_URL = urljoin(BASE, "course_query_all.queryByAllConditions")

# 查詢條件：114 學年度第 1 學期，學士班
PAYLOAD = {
    "qEdu": "U",
    "qCollege": "",
    "qdept": "",
    "qYear": "114",
    "qTerm": "1",
    "qGrade": "",
    "qClass": "",
    "cour": "",
    "teach": "",
    "qMemo": "",
    "week": "",
    "seq1": "A",
    "seq2": "M",
}

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://sea.cc.ntpu.edu.tw",
    "Referer": urljoin(BASE, "course_query_all.CHI_query_Common"),
}


# ===== 輔助函數 =====
def td_text(td, sep="|"):
    return td.get_text(separator=sep, strip=True).replace("\xa0", "").strip()


def split_name_and_note(raw_text):
    key = "備註："
    if key in raw_text:
        name, note = raw_text.split(key, 1)
        return name.strip(), note.strip()
    return raw_text.strip(), ""


def parse_name_cell(td):
    links = td.find_all("a", href=True)
    syllabus_url, limit_url = None, None
    for a in links:
        href = a["href"]
        if "queryGuide2" in href:
            syllabus_url = urljoin(BASE, href)
        if "bylimit_checkshow" in href:
            limit_url = urljoin(BASE, href)

    raw = td.get_text(separator="\n", strip=True)
    name_text, note_text = split_name_and_note(raw)
    name_lines = [l for l in name_text.splitlines() if l.strip()]
    course_name = " / ".join(name_lines) if name_lines else ""

    return course_name, note_text, syllabus_url, limit_url


def fetch_limit_table(sess, url):
    """抓取限制頁 → 回傳 dict"""
    try:
        res = sess.get(url, headers=HEADERS, timeout=30)
        res.encoding = "big5"
        soup = BeautifulSoup(res.text, "html.parser")
        table = soup.find("table")
        if not table:
            return {}
        restrictions = {}
        for tr in table.find_all("tr"):
            tds = tr.find_all("td")
            if len(tds) == 2:
                key = tds[0].get_text(strip=True)
                if key == "群組":  # 忽略群組
                    continue
                val = tds[1].get_text(strip=True)
                restrictions[key] = val
        return restrictions
    except Exception as e:
        print(f"抓限制失敗 {url}: {e}")
        return {}


def ensure_db():
    conn = sqlite3.connect("ntpu_courses.db")
    cur = conn.cursor()
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS courses (
        yearterm TEXT,
        serial TEXT,
        dept TEXT,
        grade TEXT,
        required TEXT,
        name TEXT,
        note TEXT,
        teacher TEXT,
        category TEXT,
        credit INTEGER,
        hours INTEGER,
        language TEXT,
        schedule TEXT,
        addable TEXT,
        add_limit INTEGER,
        total_limit INTEGER,
        enrolled INTEGER,
        syllabus_url TEXT,
        limit_url TEXT,
        limits_json TEXT,  -- 新增 JSON 欄位
        PRIMARY KEY (yearterm, serial)
    );
    """)
    conn.commit()
    return conn


# ===== 主流程 =====
def main():
    sess = requests.Session()
    res = sess.post(LIST_URL, headers=HEADERS, data=PAYLOAD, timeout=60)
    res.encoding = "big5"
    soup = BeautifulSoup(res.text, "html.parser")

    # 找到最大表格（課程清單）
    table = max(soup.find_all("table"), key=lambda t: len(t.find_all("tr")))
    rows = table.find_all("tr")[1:]

    conn = ensure_db()
    cur = conn.cursor()

    count = 0
    with_limits = 0

    for tr in rows:
        tds = tr.find_all("td")
        if len(tds) < 18:
            continue

        year = td_text(tds[1])
        term = td_text(tds[2])
        serial = td_text(tds[3])
        yearterm = year + term

        dept = td_text(tds[4])
        grade = td_text(tds[5], sep="|")
        required = td_text(tds[6], sep="|")
        name, note, syllabus_url, limit_url = parse_name_cell(tds[7])
        teacher = td_text(tds[8], sep="|")
        category = td_text(tds[9])
        credit = td_text(tds[10])
        hours = td_text(tds[11])
        language = td_text(tds[12])
        schedule = td_text(tds[13], sep="; ")
        addable = td_text(tds[14])
        add_limit = td_text(tds[15])
        total_limit = td_text(tds[16])
        enrolled = td_text(tds[17])

        # 抓限制
        limits_dict = {}
        if limit_url:
            limits_dict = fetch_limit_table(sess, limit_url)
            if limits_dict:
                with_limits += 1
            time.sleep(0.5)  # 禮貌 delay

        # 存課程（含 JSON 格式限制）
        cur.execute("""
        INSERT OR REPLACE INTO courses (
            yearterm, serial, dept, grade, required, name, note, teacher,
            category, credit, hours, language, schedule, addable, add_limit,
            total_limit, enrolled, syllabus_url, limit_url, limits_json
        ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        """, (
            yearterm, serial, dept, grade, required, name, note, teacher,
            category, credit, hours, language, schedule, addable, add_limit,
            total_limit, enrolled, syllabus_url, limit_url,
            json.dumps(limits_dict, ensure_ascii=False)
        ))

        count += 1

    conn.commit()
    conn.close()
    print(f"完成：匯入 {count} 筆課程，其中 {with_limits} 門有課程限制。")


if __name__ == "__main__":
    main()

完成：匯入 1370 筆課程，其中 581 門有課程限制。
